# Modelos de Machine Learning

In [ ]:
from tsbook.datasets.retail import SyntheticRetail
from sktime.utils.plotting import plot_series
from sktime.forecasting.naive import NaiveForecaster

dataset = SyntheticRetail("univariate")
y_train, X_train, y_test, X_test = dataset.load(
    "y_train", "X_train", "y_test", "X_test"
)

X_train

In [ ]:
from tsbook.forecasting.reduction import ReductionForecaster
from sklearn.ensemble import RandomForestRegressor
from sktime.transformations.series.difference import Differencer

regressor = RandomForestRegressor(n_estimators=100, random_state=42)
model = Differencer() * ReductionForecaster(
    regressor,
    window_length=30,
    steps_ahead=1,
)

model.fit(y_train, X=X_train)
y_pred = model.predict(fh=y_test.index, X=X_test)

In [ ]:
plot_series(
    y_train, y_test, y_pred, labels=["Treino", "Teste", "Previsão com ML + Diferença"]
)

In [ ]:
import numpy as np
from typing import Callable, Tuple

def mean_window_normalizer(window: np.ndarray) -> Tuple[
    Callable[[np.ndarray], np.ndarray],
    Callable[[np.ndarray], np.ndarray],
]:
    """
    Per-window normalizer: divide by the mean of the y-lag window.

    Given the current lag window (1D ndarray in feature order: [y_t, y_{t-1}, ...]),
    returns (transform, inverse_transform) where:
      transform(a)        = a / mean(window)
      inverse_transform(a)= a * mean(window)

    Notes
    -----
    - If the mean is NaN, inf, or ~0, falls back to 1.0 to avoid division by zero.
    - Exogenous X is untouched; this normalizes only the y-derived features/target.

    Parameters
    ----------
    window : np.ndarray
        1D array of lagged y values (most recent first).

    Returns
    -------
    transform : Callable[[np.ndarray], np.ndarray]
    inverse_transform : Callable[[np.ndarray], np.ndarray]
    """
    w = np.asarray(window, dtype=float).ravel()
    mu = float(np.mean(w)) if w.size else 1.0

    # robust fallback if mean is invalid or too close to zero
    if not np.isfinite(mu) or abs(mu) < 1e-12:
        mu = 1.0

    def transform(arr: np.ndarray) -> np.ndarray:
        a = np.asarray(arr, dtype=float).ravel()
        return a / mu

    def inverse_transform(arr: np.ndarray) -> np.ndarray:
        a = np.asarray(arr, dtype=float).ravel()
        return a * mu

    return transform, inverse_transform

model =  ReductionForecaster(
    regressor,
    window_length=30,
    steps_ahead=1,
    normalization_strategy=mean_window_normalizer,
)

model.fit(y_train, X=X_train)
y_pred = model.predict(fh=y_test.index, X=X_test)

In [ ]:
plot_series(
    y_train, y_test, y_pred, labels=["Treino", "Teste", "Previsão com ML + Diferença + Normalização"]
)

In [ ]:
from tsbook.forecasting.global_reduction import GlobalReductionForecaster
from typing import Optional

def mean_window_normalizer():
    """
    Row-wise normalizer factory: divide by the mean of the lag window.

    Returns a function that, given a lag window (most recent first), produces:
      transform(y_lags, y_target) -> (y_lags / m, y_target / m)
      inverse_transform(y_hat_n)  -> y_hat_n * m

    If the mean is non-finite or zero, falls back to m=1.0.
    """
    def strategy(window: np.ndarray):
        w = np.asarray(window, dtype=float)
        m = np.nanmean(w)
        if not np.isfinite(m) or m == 0:
            m = 1.0

        def transform(y_lags: np.ndarray, y_target: Optional[float]):
            y_lags_n = y_lags / m
            y_tgt_n = None if y_target is None else y_target / m
            return y_lags_n, y_tgt_n

        def inverse_transform(y_hat_n: float):
            return y_hat_n * m

        return transform, inverse_transform

    return strategy

def last_obs_normalizer():
    """
    Row-wise normalizer factory: divide by the mean of the lag window.

    Returns a function that, given a lag window (most recent first), produces:
      transform(y_lags, y_target) -> (y_lags / m, y_target / m)
      inverse_transform(y_hat_n)  -> y_hat_n * m

    If the mean is non-finite or zero, falls back to m=1.0.
    """
    def strategy(window: np.ndarray):
        w = np.asarray(window, dtype=float)
        m = w[-1]
        if not np.isfinite(m) or m == 0:
            m = 1.0

        def transform(y_lags: np.ndarray, y_target: Optional[float]):
            y_lags_n = y_lags / m
            y_tgt_n = None if y_target is None else y_target / m
            return y_lags_n, y_tgt_n

        def inverse_transform(y_hat_n: float):
            return y_hat_n * m

        return transform, inverse_transform

    return strategy


model =  GlobalReductionForecaster(
    regressor,
    window_length=30,
    steps_ahead=12,
    normalization_strategy=mean_window_normalizer,
)

model.fit(y_train, X=X_train)
y_pred = model.predict(fh=y_test.index, X=X_test)

In [ ]:
plot_series(
    y_train,
    y_test,
    y_pred,
    labels=["Treino", "Teste", "Previsão com ML + Diferença + Normalização"],
)